# Important notice
- Run this notebook in venv

### Import dependencies

In [ ]:
# %pip install --upgrade pip --quiet && pip install -r ../../requirements.txt --quiet

In [ ]:
import os
import pandas as pd
import plotly.graph_objs as go
from dotenv import load_dotenv
from plotly.subplots import make_subplots

In [ ]:
load_dotenv()

DATA_DIR = os.getenv("DATA_DIR")

### Load data

In [ ]:
def load_train_df() -> pd.DataFrame:
    return pd.read_csv(f"{DATA_DIR}/raw/train.csv", dtype={
        "img": int, # Image ID
        "x": int, # X coordinate of the top-left corner of the bounding box
        "y": int, # Y coordinate of the top-left corner of the bounding box
        "w": int, # Width of the bounding box
        "h": int, # Height of the bounding box
        "rot": int, # Whether the image shows a rotten pear (1) or not (0)
    }, index_col=0).rename(columns={
        "img": "image_id",
        "w": "x2",
        "h": "y2",
        "rot": "is_rotten",
    })

def load_test_df() -> pd.DataFrame:
    return pd.read_csv(f"{DATA_DIR}/raw/test.csv", dtype={
        "img": int, # Image ID
        "x": int, # X coordinate of the top-left corner of the bounding box
        "y": int, # Y coordinate of the top-left corner of the bounding box
        "w": int, # Width of the bounding box
        "h": int, # Height of the bounding box
    }, index_col=0).rename(columns={
        "img": "image_id",
        "w": "x2",
        "h": "y2",
    })

In [ ]:
train_df = load_train_df()
test_df = load_test_df()

### Data understanding

In [ ]:
def add_dimensions(df: pd.DataFrame) -> None:
    df["width"] = df["x2"] - df["x"]
    df["height"] = df["y2"] - df["y"]
    df["area"] = df["width"] * df["height"]

In [ ]:
add_dimensions(train_df)
add_dimensions(test_df)

In [ ]:
# Create subplots with two rows and two columns
fig = make_subplots(rows=2, cols=2, subplot_titles=("Training Set (Width)", "Training Set (Height)", "Test Set (Width)", "Test Set (Height)"))

# Add histograms for width and height for training set
fig.add_trace(go.Histogram(x=train_df["width"], name="Width (train)", opacity=0.7, nbinsx=50, histnorm="probability"), row=1, col=1)
fig.add_trace(go.Histogram(x=train_df["height"], name="Height (train)", opacity=0.7, nbinsx=50, histnorm="probability"), row=1, col=2)

# Add histograms for width and height for test set
fig.add_trace(go.Histogram(x=test_df["width"], name="Width (test)", opacity=0.7, nbinsx=50, histnorm="probability"), row=2, col=1)
fig.add_trace(go.Histogram(x=test_df["height"], name="Height (test)", opacity=0.7, nbinsx=50, histnorm="probability"), row=2, col=2)

# Update xaxis and yaxis titles
fig.update_xaxes(title_text="Width", row=1, col=1)
fig.update_xaxes(title_text="Height", row=1, col=2)
fig.update_xaxes(title_text="Width", row=2, col=1)
fig.update_xaxes(title_text="Height", row=2, col=2)
fig.update_yaxes(title_text="Probability", row=1, col=1)
fig.update_yaxes(title_text="Probability", row=1, col=2)
fig.update_yaxes(title_text="Probability", row=2, col=1)
fig.update_yaxes(title_text="Probability", row=2, col=2)

# Update layout
fig.update_layout(title="Distribution of the width and height of the bounding boxes in the training and test sets", showlegend=False)

# Show the figure
fig.show()

In [ ]:
# Create subplots with two rows and two columns
fig = make_subplots(rows=2, cols=2, subplot_titles=("Non-rotten Pears (Width)", "Non-rotten Pears (Height)", "Rotten Pears (Width)", "Rotten Pears (Height)"))

# Add histograms for width and height for non-rotten pears
fig.add_trace(go.Histogram(x=train_df[train_df["is_rotten"] == 0]["width"], name="Width (non-rotten)", opacity=0.7, nbinsx=50, histnorm="probability"), row=1, col=1)
fig.add_trace(go.Histogram(x=train_df[train_df["is_rotten"] == 0]["height"], name="Height (non-rotten)", opacity=0.7, nbinsx=50, histnorm="probability"), row=1, col=2)

# Add histograms for width and height for rotten pears
fig.add_trace(go.Histogram(x=train_df[train_df["is_rotten"] == 1]["width"], name="Width (rotten)", opacity=0.7, nbinsx=50, histnorm="probability"), row=2, col=1)
fig.add_trace(go.Histogram(x=train_df[train_df["is_rotten"] == 1]["height"], name="Height (rotten)", opacity=0.7, nbinsx=50, histnorm="probability"), row=2, col=2)

# Update xaxis and yaxis titles
fig.update_xaxes(title_text="Width", row=1, col=1)
fig.update_xaxes(title_text="Height", row=1, col=2)
fig.update_xaxes(title_text="Width", row=2, col=1)
fig.update_xaxes(title_text="Height", row=2, col=2)
fig.update_yaxes(title_text="Probability", row=1, col=1)
fig.update_yaxes(title_text="Probability", row=1, col=2)
fig.update_yaxes(title_text="Probability", row=2, col=1)
fig.update_yaxes(title_text="Probability", row=2, col=2)

# Update layout
fig.update_layout(title="Distribution of the width and height of the bounding boxes in the non-rotten and rotten pears in the training set", showlegend=False)

# Show the figure
fig.show()

In [ ]:
# Create subplots with two rows and two columns
fig = make_subplots(rows=1, cols=2, subplot_titles=("Non-rotten Pears", "Rotten Pears"))

# Add histograms for area for non-rotten pears
fig.add_trace(go.Histogram(x=train_df[train_df["is_rotten"] == 0]["area"], name="Area (non-rotten)", opacity=0.7, nbinsx=50, histnorm="probability"), row=1, col=1)

# Add histograms for area for rotten pears
fig.add_trace(go.Histogram(x=train_df[train_df["is_rotten"] == 1]["area"], name="Area (rotten)", opacity=0.7, nbinsx=50, histnorm="probability"), row=1, col=2)

# Set x-axis range and y-axis range to the max values
max_area = max(train_df["area"])
max_prob = 0.16

fig.update_xaxes(range=[0, max_area], row=1, col=1)
fig.update_xaxes(range=[0, max_area], row=1, col=2)
fig.update_yaxes(range=[0, max_prob], row=1, col=1)
fig.update_yaxes(range=[0, max_prob], row=1, col=2)

# Update xaxis and yaxis titles
fig.update_xaxes(title_text="Area", row=1, col=1)
fig.update_xaxes(title_text="Area", row=1, col=2)
fig.update_yaxes(title_text="Probability", row=1, col=1)
fig.update_yaxes(title_text="Probability", row=1, col=2)

# Update layout
fig.update_layout(title="Difference in area between rotten and non-rotten pears in the training set", showlegend=True)

# Show the figure
fig.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate the correlation matrix
corr_matrix = train_df.corr()

# Plot the heatmap
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm")

plt.show()